# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import Geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [4]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Limulunga,ZM,-15.0969,23.1376,66.22,92,65,5.03,light rain
1,1,Tres Arroyos,AR,-38.3739,-60.2798,73.60,22,0,8.46,clear sky
2,2,Tikrit,IQ,34.6071,43.6782,55.92,56,5,6.78,clear sky
3,3,San Buenaventura,MX,27.0833,-101.5333,71.47,68,0,5.50,clear sky
4,4,Aykhal,RU,66.0000,111.5000,-16.64,92,100,4.00,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [5]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 92


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [6]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]


# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
6,6,Kapaa,US,22.0752,-159.3190,82.38,74,100,7.00,overcast clouds
13,13,Atuona,PF,-9.8000,-139.0333,78.04,69,34,13.02,scattered clouds
16,16,Pandan Niog,PH,6.2282,120.5631,81.50,76,100,6.67,overcast clouds
22,22,Butaritari,KI,3.0707,172.7902,81.75,77,100,16.55,overcast clouds
27,27,Touros,BR,-5.1989,-35.4608,78.55,81,95,11.86,overcast clouds
30,30,Hilo,US,19.7297,-155.0900,78.53,77,75,10.36,light rain
55,55,Viedma,AR,-40.8135,-62.9967,79.29,19,99,9.82,overcast clouds
58,58,Kavieng,PG,-2.5744,150.7967,83.32,74,89,5.64,light rain
64,64,Sao Borja,BR,-28.6606,-56.0044,79.95,30,0,7.34,clear sky
66,66,Hithadhoo,MV,-0.6000,73.0833,82.15,76,100,18.59,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [7]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
preferred_cities_df.count()

City_ID            100
City               100
Country            100
Lat                100
Lng                100
Max Temp           100
Humidity           100
Cloudiness         100
Wind Speed         100
Current Weather    100
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp","Current Weather", "Lat", "Lng"]].copy()
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng
6,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190
13,Atuona,PF,78.04,scattered clouds,-9.8000,-139.0333
16,Pandan Niog,PH,81.50,overcast clouds,6.2282,120.5631
22,Butaritari,KI,81.75,overcast clouds,3.0707,172.7902
27,Touros,BR,78.55,overcast clouds,-5.1989,-35.4608
...,...,...,...,...,...,...
696,Kahului,US,80.62,few clouds,20.8947,-156.4700
697,Manadhoo,MV,81.66,overcast clouds,5.7667,73.3833
705,Alice Springs,AU,80.15,broken clouds,-23.7000,133.8833
706,Kloulklubed,PW,82.90,overcast clouds,7.0419,134.2556


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [9]:
# Create a new column "Hotel Name".

hotel_df.insert(loc=5,column= "Hotel Name", value = "")

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Hotel Name,Lng
6,Kapaa,US,82.38,overcast clouds,22.0752,,-159.3190
13,Atuona,PF,78.04,scattered clouds,-9.8000,,-139.0333
16,Pandan Niog,PH,81.50,overcast clouds,6.2282,,120.5631
22,Butaritari,KI,81.75,overcast clouds,3.0707,,172.7902
27,Touros,BR,78.55,overcast clouds,-5.1989,,-35.4608
30,Hilo,US,78.53,light rain,19.7297,,-155.0900
55,Viedma,AR,79.29,overcast clouds,-40.8135,,-62.9967
58,Kavieng,PG,83.32,light rain,-2.5744,,150.7967
64,Sao Borja,BR,79.95,clear sky,-28.6606,,-56.0044
66,Hithadhoo,MV,82.15,overcast clouds,-0.6000,,73.0833


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [10]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": Geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [11]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kapaa - nearest hotel: Pono Kai Resort
Atuona - nearest hotel: Pearl Resort
Pandan Niog - nearest hotel: No hotel found
Butaritari - nearest hotel: No hotel found
Touros - nearest hotel: Pousada Atlântico
Hilo - nearest hotel: Dolphin Bay Hotel
Viedma - nearest hotel: Níjar
Kavieng - nearest hotel: Kavieng Hotel
Sao Borja - nearest hotel: Hotel Obino
Hithadhoo - nearest hotel: Pebbles Inn
Namatanai - nearest hotel: Namatanai Hotel
Cabo San Lucas - nearest hotel: Comfort Rooms
Vaini - nearest hotel: Keleti International Resort
Samarai - nearest hotel: No hotel found
Finschhafen - nearest hotel: No hotel found
Bubaque - nearest hotel: Chez Julio
Avarua - nearest hotel: Paradise Inn
Hervey Bay - nearest hotel: Hervey Bay Hotel
Mitsamiouli - nearest hotel: Hotel le Paradis
Cabedelo - nearest hotel: No hotel found
Isangel - nearest hotel: Tanna Lodge
Merauke - nearest hotel: No hotel found
Itarema - nearest hotel: Pousada Oásis
Pouebo - nearest hotel: No hotel found
Ge

,City,Country,Max Temp,Current Weather,Lat,Hotel Name,Lng
6,Kapaa,US,82.38,overcast clouds,22.0752,Pono Kai Resort,-159.3190
13,Atuona,PF,78.04,scattered clouds,-9.8000,Pearl Resort,-139.0333
16,Pandan Niog,PH,81.50,overcast clouds,6.2282,No hotel found,120.5631
22,Butaritari,KI,81.75,overcast clouds,3.0707,No hotel found,172.7902
27,Touros,BR,78.55,overcast clouds,-5.1989,Pousada Atlântico,-35.4608
...,...,...,...,...,...,...,...
696,Kahului,US,80.62,few clouds,20.8947,Maui Seaside Hotel,-156.4700
697,Manadhoo,MV,81.66,overcast clouds,5.7667,No hotel found,73.3833
705,Alice Springs,AU,80.15,broken clouds,-23.7000,Aurora Alice Springs,133.8833
706,Kloulklubed,PW,82.90,overcast clouds,7.0419,The Adventures Inn,134.2556


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [12]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Hotel Name,Lng
6,Kapaa,US,82.38,overcast clouds,22.0752,Pono Kai Resort,-159.3190
13,Atuona,PF,78.04,scattered clouds,-9.8000,Pearl Resort,-139.0333
16,Pandan Niog,PH,81.50,overcast clouds,6.2282,No hotel found,120.5631
22,Butaritari,KI,81.75,overcast clouds,3.0707,No hotel found,172.7902
27,Touros,BR,78.55,overcast clouds,-5.1989,Pousada Atlântico,-35.4608


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [13]:
# Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [14]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Country","Current Weather"]
)



# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Weather)